In [1]:
from os import listdir
from os.path import isfile, join
import re
import csv
import gensim
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
import logging
import os
import pandas as pd 
import sys
import sklearn.metrics
import sklearn
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer
import random
import math
import warnings
import utils
from utils import *
from utils2 import *
import scipy.stats
from dataDictionariesLexicons import *
from testData import *
from polesConstructs import *

random.seed(1)

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
# warnings.filterwarnings('ignore')

sys.path.insert(0, '../') # necessary in order to import word2vecTraining where the epoch saver class, which is part of the model, is located

In [2]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
# modelFilePath = os.path.join(modelsPath, 'GoogleNews-vectors-negative300.bin')
# model2 = KeyedVectors.load_word2vec_format(modelFilePath,binary=True) 
modelFilePath = os.path.join(modelsPath, 'GoogleNews-vectors-negative300LowerCase.bin')
model1 = KeyedVectors.load(modelFilePath) 

In [3]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.6B.300d.txt.word2vec.bin')
model2 = KeyedVectors.load_word2vec_format(modelFilePath,binary=True) 

In [4]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.twitter.27B.200d.txt.word2vec.bin')
model3 = KeyedVectors.load(modelFilePath) 

In [5]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.42B.300d.txt.word2vec.bin')
model4 = KeyedVectors.load(modelFilePath) 

In [6]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.840B.300d.txt.word2vec.bin')
model5 = KeyedVectors.load(modelFilePath) 

In [7]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'fastext-common-crawl-300d-2M-subword.bin')
model6 = KeyedVectors.load(modelFilePath)

In [8]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'fastext-wiki-news-300d-1M-subword-Original.bin')
model7 = KeyedVectors.load(modelFilePath)

In [9]:
# models = [model1,model2,model3,model4]
# models = [model1,model2]
models = [model1,model2,model3,model4,model5,model6,model7]
# models = [model1]

In [10]:
#change default lexicons
# lexicons = [harvardGeneralInquirer3623, vaderLexicon7062,NRCEmotionLexicon5555,opinionLexicon6786,afinnLexicon2477]
# lexicons = [WEAT1]
lexicons = [harvardGeneralInquirer3623]
dataDict = dataDictLoadLexicons(dataDict,lexicons)
# WEAT1 harvardGeneralInquirer3623

In [11]:
dataSetIndexes = [
# '0-1', #percentage females per occupation
# '0-2', #country's GDP per capita
# '0-3', #car brands and prices
# '0-4', #voting behavior by ethnic/racial group
# '0-5', #voting behavior by profession
    
'1-1', #gender
'1-2',
'1-3',
'1-4',
'1-5',
    
'2-1', #Race/ethnicity
'2-2',
'2-3',
'2-4',    
  
'3-1', #Sexual orientation  
    
'4-1', #religiosity    
'4-2', #religion Islam and Christianity
    
'5-1', #age 
'6-1', #socioeconomic status
'7-1', #physical beauty
    
'8-1', #Personal ideology
'8-2', #Party affiliation and political parties
'8-3', #US presidents
'8-4', #Ideologies abstract
'8-5', #influential conservatives and liberals
]

In [12]:
rows = len(dataSetIndexes)
columns = len(models)
outputMatrix = np.zeros((rows,columns))
outputMatrixPvalues = np.zeros((rows,columns))

## Dimensions against models

In [13]:
#Checking for the existence of uppercase letters in models
# [w for w in model6.wv.vocab if any(x.isupper() for x in w)]

In [14]:
#Checking for the existence _ in models
# [w for w in model2.wv.vocab if '-' in w]

In [15]:
random.seed(0)
outputMatrices = []
for outputMatrixIndex in range(500):
    print("outputMatrixIndex",outputMatrixIndex)
    for rowNumber, dataSetIndex in enumerate(dataSetIndexes):
        print(dataDict[dataSetIndex]['name'])
        modelsCorrelations=[]
        modelspvalues=[]
        for i,model in enumerate(models):    
    #         print("model", i+1)
            correlations = []
            pvalues = []
            constructPole1 = dataDict[dataSetIndex]['constructPole1']
            constructPole2 = dataDict[dataSetIndex]['constructPole2']

            lengthPole1 = len(constructPole1)
            lengthPole2 = len(constructPole2)
            chunkSize=3
            if len(constructPole1) > 2:
                constructPole1 = random.sample(constructPole1,k=chunkSize*(max(1,round(lengthPole1/4))))
            else:
                if chunkSize <= 2:
                    constructPole1 = random.sample(constructPole1,k=1)
            if len(constructPole2) > 2:
                constructPole2 = random.sample(constructPole2,k=chunkSize*(max(1,round(lengthPole2/4))))
            else:
                if chunkSize <= 2:
                    constructPole2 = random.sample(constructPole2,k=1)
                
            if i>=1: #Glove and fastext vocabulary uses - instead of _ So we need to alter Construct poles and real data - Checked!
                constructPole1,constructPole2,RealDataTemp=replaceUnderscoreForDash(constructPole1,constructPole2,RealDataTemp)
            if i in [1,2,3]: # Lowercase poles for models With only lowercase vocabularies - Checked!
                constructPole1 = list(set(list(map(str.lower,constructPole1))))
                constructPole2 = list(set(list(map(str.lower,constructPole2))))
            constructPole1,constructPole2 = constructsFilter(model,constructPole1,constructPole2,printFlag=False)# Filter out terms not in model vocabulary

            for RealDataTemp in dataDict[dataSetIndex]['RealDataLexicons']:
    #         for RealDataTemp in [voteByEthnicity]:
                RealData = realDataFilter(model,RealDataTemp,0)#Filter out terms not in model vocabulary (Last parameter Flag prints out OOV words)
                Axis = dimensionN(model, constructPole1,constructPole2)  
                AxisName='Axis'
                try:
                    dataFrame = makeDF(model, RealData.keys(),Axis,AxisName)
                except:
                    continue
                dataFrame['RealValues']=RealData.values()
                spearmanCorr,pearsonCorr=calculateCorrelations(dataFrame,printFlag=False)
    #             correlations.append(pearsonCorr[0])
    #             pvalues.append(pearsonCorr[1])
                correlations.append(spearmanCorr[0])
                pvalues.append(spearmanCorr[1])            
    #         print("correlations for model", i+1)
    #         print(*correlations)
    #         print("mean: ", np.array(correlations).mean())
#             modelsCorrelations.append(np.array(correlations).mean())
            modelsCorrelations.append(np.nanmean(correlations))
#             modelspvalues.append(np.array(pvalues).mean())#This can only be done if only one lexicon is being tested
            modelspvalues.append(np.nanmean(pvalues))#This can only be done if only one lexicon is being tested
    #         print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        print(*modelsCorrelations)
        print(np.array(modelsCorrelations).mean())
        outputMatrix[rowNumber]=modelsCorrelations
        outputMatrixPvalues[rowNumber]=modelspvalues
        #Print closest neighbors to axis Poles
    #     print([w[0] for w in model.wv.similar_by_vector(-Axis,restrict_vocab=50000,topn=50)])
    #     print([w[0] for w in model.wv.similar_by_vector(Axis,restrict_vocab=50000,topn=50)])

        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    outputMatrices.append(outputMatrix)
# outputMatrix=np.array(outputMatrices).mean(axis=0) #shape of outputMatrices=(2,20,7)    
outputMatrix=np.nanmean(np.array(outputMatrices),axis=0) #shape of outputMatrices=(2,20,7)    

outputMatrixIndex 0
gender general
0.21596582529642516 0.1546834945909133 0.15929959790102197 0.246449193940538 0.24838487274514673 0.32006960032909865 0.20041952487248244
0.22075315852508945
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender family roles
0.008484115354012367 0.07416644209617275 0.02663341710930318 0.09539520819622692 0.11901326707652235 0.20667916906938577 0.09915742758672
0.08993272092690618
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and young age
0.10788435045954331 0.1181680237172067 0.05741047682862865 0.20577376678618597 0.17691431273215272 0.1883581179994758 0.17112579516652687
0.14651926338424573
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender masculinity and femininity
0.2373733719609817 0.24951690530340281 0.029122093049238705 0.3323625671516852 0.2718485687185625 0.16959290729524767 0.0799924208074061
0.1956869763266464
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and popular given names 
0.025211832895136725 -0.1

C:\Users\drozado\Sync\workspace\AlgorithmicBias\AlgorithmicBiasInWordEmbeddings\analysis\utils.py:15: RuntimeWarning: invalid value encountered in double_scalars
  normalized_vector = vector / np.linalg.norm(vector)
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: Mean of empty slice
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice


-0.16321443128127502 -0.15222548932717705 nan 0.22293342053004342 0.3878242515340997 0.39378568579162726 0.10140861138574658
nan
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
influential conservatives and liberals
0.20860339048870527 0.1919733907143434 0.20231797743588653 0.18157671347316542 0.2349804038487721 0.08694131226635196 0.10051696080192962
0.1724157355755935
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
outputMatrixIndex 1
gender general
0.15381245554533887 0.11836311205429778 -0.033731548781518655 0.13850946245373666 0.08812679838024565 0.18276074568478112 0.13493734719343387
0.11182548179004503
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender family roles
0.08182616832876786 0.13600621144570246 0.03172623178011051 0.15814506516207763 0.05938431829370946 0.13421975641837758 0.046202740179309094
0.09250149880115066
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and young age
0.10788435045954331 0.1181680237172067 0.05741047682862865 0.20577376678

0.21327330984068177 0.11958483929237936 0.13746241456816036 0.374799013308606 0.4178219317551023 0.1785097322616134 0.04528034301700011
0.21239022629193477
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
physical beauty
0.6056997353303905 0.4403467118981127 0.11372858509594831 0.4982295165093321 0.5027113820858973 0.6238619779055796 0.568287020168094
0.47898070414190785
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
personal ideology
0.2164781879644344 0.2418599575316062 0.23034996590010914 0.1897766420074794 0.4110240884210153 0.40333351639232484 0.3208145455580002
0.28766241482499566
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
party affiliation and political parties
0.06774623827982289 0.08850543487236874 0.030302756685969545 0.08464642225142921 0.1501219328982729 0.20483419828067956 0.1896058217922652
0.11653754358011541
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
U.S. presidents
0.036749501273459946 0.038611862120348554 0.22136933794037367 0.05924117206864457 0

-0.08481088852023531 -0.11166489950815053 -0.020470873641539353 -0.05822953419820204 -0.08111103096894978 0.11022179002233856 0.14115991865881916
-0.014986502593702758
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
sexual orientation
-0.15477424039080842 0.09203427031787297 0.029949576384282087 0.15304147493635 -0.06413912312776776 -0.25103480793548355 -0.09647846539973644
-0.04162875931647016
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
religiosity
0.04388347534201042 -0.19628958544296946 -0.07602941786117788 -0.25036408297370283 -0.1577904043656482 -0.09330528702681691 -0.0019672458899332003
-0.10455179260260546
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
religion Christianity and Islam
-0.20824878087514617 -0.32214258067100243 -0.030481610813106144 -0.35855191676374126 -0.3514250507897085 -0.2065505122522335 -0.25897703979680553
-0.24805392742310625
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
age
0.1805635191422311 0.10886036993295138 0.1883305659044727 0.201

...

0.3366688847341558 -0.1805025198442216 nan -0.0870200857745201 0.16204953868903405 0.41853744233629275 0.23718707784660767
nan
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
influential conservatives and liberals
0.16652349607706743 0.24430233417165279 0.2398586663301562 0.2635717257946614 0.20285369062040337 0.17663753534147664 0.15526698065072395
0.2070020612837345
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
outputMatrixIndex 494
gender general
0.1912821571593279 0.023316828708794514 0.15929959790102197 0.19874464618225346 0.08812679838024565 0.32006960032909865 0.09108899235073138
0.15313266014449622
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender family roles
0.005537962975500007 0.08813412033022573 0.022964643526710033 0.014530410099496962 0.004952966212786183 0.1838252340391446 0.18112030262201514
0.0715808056865541
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and young age
0.10788435045954331 0.1181680237172067 0.05741047682862865 0.205773766786

C:\Users\drozado\Sync\workspace\AlgorithmicBias\AlgorithmicBiasInWordEmbeddings\analysis\utils.py:15: RuntimeWarning: invalid value encountered in double_scalars
  normalized_vector = vector / np.linalg.norm(vector)
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: Mean of empty slice
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice


0.24174064111407467 0.18285328041436352 nan 0.062406946982917766 0.10567022287300903 0.23426404481997257 0.0782508280203682
nan
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
influential conservatives and liberals
0.1610685610215804 0.23965470483719517 0.2498337458765341 0.3331552126242441 0.22985705089794228 0.0961165185645698 0.1869380887819698
0.21380341180057652
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
outputMatrixIndex 495
gender general
0.15869181954822417 0.023316828708794514 0.05328041807636357 0.24711685357237045 0.1288978343938761 0.10604444730774765 0.20041952487248244
0.13110967521140843
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender family roles
0.15664397581312364 0.12944057274188786 0.00552127218423267 0.16609288541941164 0.08705266756455352 0.18968152152528675 0.05134410175357051
0.11225385671458095
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and young age
0.10788435045954331 0.1181680237172067 0.05741047682862865 0.205773766786185

0.25299012222337247 0.08729071356904458 0.1078280968826602 0.3266703681521815 0.38419111488152 0.1328488406239917 0.104622831294523
0.19949172680389907
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
physical beauty
0.5475694705819975 0.3521301371049728 0.10781564502054829 0.5246832593779465 0.6790573829394342 0.4898074467357855 0.388127614763858
0.44131299378922034
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
personal ideology
0.2326402442488844 0.1988489073016591 0.09840938951025466 0.40608714603939305 0.315993689467755 0.4576811454934383 0.3760920696454517
0.29796465595811955
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
party affiliation and political parties
0.06629191033297543 0.1018792254062162 0.030302756685969545 0.08464642225142921 -0.12222257340260213 0.13638396694424998 0.027489039605785803
0.04639582111771772
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
U.S. presidents
0.07753817746282124 0.12711729699271726 0.12297183430565654 0.29140565465419993 0.115

-0.07046794741349949 -0.0873365216039969 -0.020470873641539353 -0.05822953419820204 -0.08987660074335224 0.13812590551009152 0.11434035224711958
-0.0105593171204827
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
sexual orientation
-0.11454131566790676 0.054918444727261706 0.01649419928169231 0.10023707584627618 -0.0036715938473732583 -0.29417434626434963 -0.06869372397286179
-0.044204465699608744
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
religiosity
0.1509493310974937 -0.1862225961266462 -0.1016072560239312 -0.24156032305240774 -0.21427173858014995 -0.08202779631170727 0.04848119485096046
-0.08946559773519833
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
religion Christianity and Islam
-0.20824878087514617 -0.32214258067100243 -0.030481610813106144 -0.35855191676374126 -0.3514250507897085 -0.20655077918824002 -0.25897703979680553
-0.24805396555682144
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
age
0.1525161484987934 0.22667540232199598 0.1999113706365675 0.1838

In [16]:
#print output matrix
for r in outputMatrix:
    print(*r)

0.15869181954822412 0.19845980846359515 0.07416107142548846 0.2703393909039949 0.3155249924061813 0.18426885982774938 0.1729016102250954
-0.023871065730879455 0.19639521338238816 0.0070528521463581705 0.23669602203872847 0.010736501007571958 0.13577004437277654 0.06599337286001454
0.10788435045954271 0.11816802371720507 0.057410476828628924 0.2057737667861843 0.1769143127321516 0.18835811799947458 0.17112579516652743
0.2373733719609836 0.2495169053034055 0.029122093049238636 0.332362567151685 0.27184856871856405 0.16959290729524867 0.07999242080740676
0.013900750417924651 -0.10063109118328074 -0.022700324294517547 0.0008011915581990041 0.005069406052577727 0.26422293246216105 0.007760969515218978
-0.2731257345703309 0.03910012188113428 -0.005452786901373407 -0.06321080917566828 -0.09372552498894036 0.11485503697696241 0.026212655824020107
-0.08893871583456557 -0.17564040112257706 -0.07226612326835698 -0.18390604439670316 -0.22936885817514863 -0.22384070724778454 -0.009467270584742412
-

In [17]:
#print P values output matrix
for r in outputMatrixPvalues:
    print(*r)

9.073204457528283e-22 2.9681821262420173e-33 1.1011050840169772e-05 1.6393717427475618e-61 2.7555871739146236e-84 5.950694217936787e-29 1.2863131181480324e-25
0.1518109110771282 1.3765962508395068e-32 0.6762939287780023 3.6578477364558764e-47 0.5189474662009355 2.507697198172739e-16 7.26302805462903e-05
8.293898494025836e-11 1.180725455553013e-12 0.0006703236139930866 7.956069139643043e-36 8.942764556148741e-27 3.3708333911118294e-30 4.0443684975839228e-25
2.283666279060523e-47 3.7366278776024774e-52 0.08464302059483893 7.335967386359659e-94 3.3124990486596215e-62 1.0299228841110107e-24 1.4973584199931225e-06
0.4040045344625768 1.4763431099516741e-09 0.1789465965885314 0.9616140567044638 0.7607278027993778 9.295848757057266e-59 0.6411543818844112
1.031328151566483e-62 0.019054716796487833 0.7468460024968192 0.0001441571478255045 1.6751044644875117e-08 4.412092924543154e-12 0.11538366439189644
8.818783972265759e-08 2.676915755876949e-26 1.838351051068537e-05 7.786223627527293e-29 2.5509

## adjusting P values for multiple tests

In [18]:
outputMatrixPvalues.shape

(20, 7)

In [19]:
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

stats = importr('stats')

#bonferroni most conservative, holm also valid under arbitrary assumptions
p_adjust = stats.p_adjust(FloatVector(outputMatrixPvalues.flatten()), method = 'holm')
outputMatrixPvaluesCorrected = np.array(p_adjust).reshape(outputMatrixPvalues.shape)

In [20]:
#print P values output matrix
for r in outputMatrixPvaluesCorrected:
    print(*r)

6.623439253995647e-20 2.4932729860432947e-31 0.00037437572856577225 1.9180649390146472e-59 3.389372223914987e-82 4.820062316528798e-27 9.775979697925047e-24
1.0 1.1425748881967906e-30 1.0 3.84074012327867e-45 1.0 1.6300031788122804e-14 0.0023241689774812896
4.644583156654468e-09 7.20242527887338e-11 0.018769061191806425 6.921780151489447e-34 6.975356353796018e-25 2.7640833807117e-28 3.033276373187942e-23
2.4206862558041546e-45 4.110290665362725e-50 1.0 9.390038254540363e-92 3.9087488774183536e-60 7.621429342421479e-23 5.390490311975241e-05
1.0 7.38171554975837e-08 1.0 1.0 1.0 1.0783184558186427e-56 1.0
1.2272805003641146e-60 0.47636791991219585 1.0 0.004324714434765135 7.627779524020645e-07 2.6031348254804607e-10 1.0
3.880264947796934e-06 2.061225132025251e-24 0.0006066558468526172 6.228978902021835e-27 2.5254006384995983e-42 2.870635048935479e-40 1.0
0.3093699588700927 1.0 1.0 0.5516848614444924 4.4715972784759905e-05 1.0 5.705882015004077e-16
3.451602021140001e-08 6.416136288533662e-